Importemos las librerías a usar

In [1]:
import pulp as lp
import pandas as pd
import numpy as np
import pickle
import os
import datetime
import statsmodels.api as sm
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals, value
import itertools

Llamamos los modelos predictivos a usar.

In [2]:
name_path = 'Input/name.pkl'
cebolla_path = 'Input/cebolla_cabezona.pkl'
platano_path = 'Input/platano_verde.pkl'
tomate_path = 'Input/tomate_chonto.pkl'
ajo_path = 'Input/ajo.pkl'

In [3]:
import glob
import os

# Define the folder path containing the CSV files
folder_path = 'Input/Variables/'

# Use glob to get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Initialize an empty list to store individual DataFrames
dfs = []

# Loop over the list of CSV files and read each into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)
    
    
var_sel_df = pd.concat(dfs, ignore_index=True)

In [4]:
with open(name_path, 'rb') as file:
    name_model = pickle.load(file)

with open(cebolla_path, 'rb') as file:
    cebolla_model = pickle.load(file)
    
with open(platano_path, 'rb') as file:
    platano_model = pickle.load(file)
    
with open(tomate_path, 'rb') as file:
    tomate_model = pickle.load(file)
    
with open(ajo_path, 'rb') as file:
    ajo_model = pickle.load(file)

Vamos a leer las datos de los parametros actuales de los productos.

In [5]:
parametros_base = pd.read_csv('Input\\base_de_datos.csv')

In [6]:
parametros_pivot = pd.pivot_table(parametros_base, values=['precio','Precio medio','unit cost'], index='sku', aggfunc='last').reset_index()

In [7]:
parametros_pivot.loc[parametros_pivot['Precio medio'].isnull(),'Precio medio'] = parametros_pivot.precio
parametros_pivot.loc[parametros_pivot['Precio medio']<parametros_pivot['unit cost'],'Precio medio'] = parametros_pivot.precio
parametros_pivot.loc[parametros_pivot['Precio medio']<parametros_pivot['unit cost'],'Precio medio'] = parametros_pivot['unit cost']

In [8]:
def get_variables(sku, df=var_sel_df):
    df_filtered = df[df.producto==sku]
    return list(df_filtered.sku)

In [9]:
def get_cost(_list,_df=parametros_pivot):
    new_df = _df[_df.sku.isin(_list)==True]
    new_df = new_df.set_index('sku')
    unit_cost_dict = new_df.loc[_list,'unit cost'].to_dict()
    
    return unit_cost_dict

def get_top_price(_list,_df=parametros_pivot):
    new_df = _df[_df.sku.isin(_list)==True]
    new_df = new_df.set_index('sku')
    new_df['Precio medio'] = new_df['Precio medio']*1.2
    unit_cost_dict = new_df.loc[_list,'Precio medio'].to_dict()
    
    return unit_cost_dict

Definimos los sku que predicen nuestros productos top 5.

In [10]:
precios_name = get_variables('BAQ-FRU1-CAT6-64:86:193:194')
precios_cebolla = get_variables('BAQ-FRU1-CAT104105-60271:510131:510132:258690')
precios_platano = get_variables('BAQ-FRU1-CAT1-47:67:151:152')
precios_tomate = get_variables('BAQ-FRU1-CAT104105-305509:1018259:1018260:563293')
precios_ajo = get_variables('BAQ-FRU1-CAT2-346:464:1180:1181')

Definimos los parameotros de restricción de cada producto.

In [11]:
parametros_name = parametros_pivot[parametros_pivot.sku.isin(precios_name)==True]
parametros_cebolla = parametros_pivot[parametros_pivot.sku.isin(precios_cebolla)==True]
parametros_platano = parametros_pivot[parametros_pivot.sku.isin(precios_platano)==True]
parametros_tomate = parametros_pivot[parametros_pivot.sku.isin(precios_tomate)==True]
parametros_ajo = parametros_pivot[parametros_pivot.sku.isin(precios_ajo)==True]

Cargamos los parametros de los modelos.

In [12]:
params_name = name_model.params
params_ajo = ajo_model.params
params_tomate = tomate_model.coef_
params_platano = platano_model.params
params_cebolla = cebolla_model.params

In [13]:
def generate_data(sku,sep):
    precios_sku = get_variables(sku)
    top_val = get_top_price(precios_sku)
    low_val = get_cost(precios_sku)
    
    list_val = list()
    
    for i,j in top_val.items():
        for m,n in low_val.items():
            if i == m:
                list_val.append(list(range(round(n),round(j),sep)))
                
    combined_list = [np.array([combination]) for combination in itertools.product(*list_val)]
    
    return combined_list

In [14]:
def optimization(sku,sep,model,t=0):
    entry_np = generate_data(sku,sep)
    cost_i = list(get_cost(get_variables(sku)).values())[0]
    df = pd.DataFrame()
    cantidad = []
    contribucion = []
    
    
    if t == 0:
        for i in entry_np:
            x = np.hstack([np.ones((i.shape[0], 1)), i])
            q = model.predict(x)
            cantidad.append(q[0])
            contribucion.append(q[0]*(i[0][0]-cost_i))
    elif t == 1:
        for i in entry_np:
            q = model.predict(i)
            cantidad.append(q[0])
            contribucion.append(q[0]*(i[0][0]-cost_i))
    elif t == 2:
        for i in entry_np:
            q = model.predict(i)
            q_i = np.exp(q[0])
            cantidad.append(q_i)
            contribucion.append(q_i*(i[0][0]-cost_i))
    
    df['parametros'] = entry_np
    df['cantidad'] = cantidad
    df['contribucion'] = contribucion
    
    return df[df.contribucion==max(df.contribucion)]

In [15]:
data_platano = generate_data('BAQ-FRU1-CAT1-47:67:151:152',500)

In [16]:
opt_platano = optimization('BAQ-FRU1-CAT1-47:67:151:152',500,platano_model)

In [17]:
data_tomate = generate_data('BAQ-FRU1-CAT104105-305509:1018259:1018260:563293',50)

In [18]:
opt_tomate = optimization('BAQ-FRU1-CAT104105-305509:1018259:1018260:563293',50,tomate_model,1)

In [19]:
data_ajo = generate_data('BAQ-FRU1-CAT2-346:464:1180:1181',50)

In [20]:
opt_ajo = optimization('BAQ-FRU1-CAT2-346:464:1180:1181',50,ajo_model,2)

In [21]:
data_cebolla = generate_data('BAQ-FRU1-CAT104105-60271:510131:510132:258690',50)

In [22]:
opt_cebolla = optimization('BAQ-FRU1-CAT104105-60271:510131:510132:258690',50,cebolla_model)

In [23]:
data_name = generate_data('BAQ-FRU1-CAT6-64:86:193:194',200)

In [24]:
opt_name = optimization('BAQ-FRU1-CAT6-64:86:193:194',200,name_model)

In [25]:
opt_platano['sku'] = 'BAQ-FRU1-CAT1-47:67:151:152'
opt_tomate['sku'] = 'BAQ-FRU1-CAT104105-305509:1018259:1018260:563293'
opt_ajo['sku'] = 'BAQ-FRU1-CAT2-346:464:1180:1181'
opt_cebolla['sku'] = 'BAQ-FRU1-CAT104105-60271:510131:510132:258690'
opt_name['sku'] = 'BAQ-FRU1-CAT6-64:86:193:194'

In [26]:
opt_table = pd.concat([opt_ajo,opt_cebolla,opt_name,opt_platano,opt_name])

In [34]:
precio_sug = list()
for i in list(opt_table.parametros):
    precio_sug.append(i[0][0])

In [43]:
sku_sub = list()
for i in list(opt_table.sku):
    temp = get_variables(i)
    for j in temp:
        sku_sub.append(j)

In [46]:
precios_sub = list()
for i in list(opt_table.parametros):
    for j in i:
        for k in j:
            precios_sub.append(k)

In [48]:
sku_sub_df = pd.DataFrame()
sku_sub_df['sku'] = sku_sub
sku_sub_df['precio_sugerido'] = precios_sub

In [35]:
opt_table['precio_sugerido'] = precio_sug

In [50]:
new_opt_table = pd.concat([opt_table.drop(columns=['parametros']),sku_sub_df])

In [53]:
new_opt_table = new_opt_table[new_opt_table.sku.duplicated()==False]

In [54]:
data_real = pd.pivot_table(parametros_base, index=['fecha','sku','producto'], values = ['cantidad','totalContribucionSinDescuento'],aggfunc='sum').reset_index()

In [55]:
data_real = pd.pivot_table(data_real,index=['sku'],values = ['cantidad','totalContribucionSinDescuento'], aggfunc='mean').reset_index()

In [59]:
sku_data = parametros_base[['sku','producto']]
sku_data.drop_duplicates(inplace=True)

C:\Users\crama\AppData\Local\Temp\ipykernel_2608\3189512080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_data.drop_duplicates(inplace=True)


In [57]:
to_export = pd.merge(new_opt_table,data_real, on='sku', how='left')

In [61]:
to_export = pd.merge(to_export, sku_data, on='sku', how='left')

In [65]:
replacements = {'cantidad_x':'Cantidad Estimada',
               'contribucion':'Contribución Estimada',
               'precio_sugerido':'Precio Sugerido',
               'cantidad_y':'Cantidad Real',
               'totalContribucionSinDescuento':'Contribución Real',
               'producto':'Producto'}

to_export.rename(columns=replacements, inplace=True)

In [67]:
to_export['Diff Cantidad'] = to_export['Cantidad Estimada']/to_export['Cantidad Real'] -1
to_export['Diff Contribución'] = to_export['Contribución Estimada']/to_export['Contribución Real'] -1

In [68]:
to_export = to_export[['Producto','Precio Sugerido','Cantidad Estimada','Cantidad Real','Diff Cantidad',
                       'Contribución Estimada','Contribución Real','Diff Contribución']]

In [70]:
to_export.to_csv('../Dashboard/data/resultados.csv')